In [1]:
import re
import requests
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from tqdm import tqdm

# page nos. 1 to 708
hindi_url = 'https://dsal.uchicago.edu/cgi-bin/app/bahri_query.py?page='

In [2]:
soups = {}

def get_soup(page):
    if page in soups:
        return soups[page]
    
    res = requests.get(hindi_url + str(page))
    soup = BeautifulSoup(res.text)
    soups[page] = soup
    return soup

In [3]:
for i in tqdm(range(1,709)):
    get_soup(i)

100%|██████████| 708/708 [06:03<00:00,  1.95it/s]


In [4]:
import pickle
pickle.dump(soups, open('./hindi_soups.pickle', 'wb'))

RecursionError: maximum recursion depth exceeded

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [20]:
def process_entry(div):
    return {
        'headword': div.select_one('hw').text.strip(),
        'subentry_count': len(div.select('b')) + len([c for c in div.text.strip() if c == '~']),
        'html': str(div)
    }

sorted_entries = pd.concat([pd.DataFrame([process_entry(div) for div in soups[page].select('div.hw_result > div')]) for page in soups.keys()], ignore_index=True).sort_values('subentry_count', ascending=False)

In [21]:
sorted_entries[:20]

headword  subentry_count  \
10263     हाथ hātha              99   
9746       सिर sira              82   
7757    मुँह mum̐ha              78   
4737       दिल dila              59   
1472    करना karanā              54   
1334        और aura              50   
1230         एक ēka              49   
6860       बात bāta              49   
8370   लगाना lagānā              46   
3629          जी jī              43   
4889      देना dēnā              43   
2623       घर ghara              41   
1213      ऊपर ūpara              41   
1732      कुछ kucha              37   
4728       दिन dina              33   
9662      साथ sātha              33   
5276       नाम nāma              33   
593         आगे āgē              32   
552      आँख ām̐kha              31   
5228       नाक nāka              31   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [28]:
len(sorted_entries)

10391

In [27]:
sorted_entries.subentry_count.value_counts().sort_index(ascending=False).pipe(lambda x: x/x.sum()).cumsum()[-20:]

21    0.006159
20    0.007506
19    0.008854
18    0.010297
17    0.012318
16    0.013954
15    0.016841
14    0.020787
13    0.025310
12    0.032624
11    0.042344
10    0.058031
9     0.080839
8     0.112982
7     0.166779
6     0.249350
5     0.389183
4     0.595227
3     0.797902
2     1.000000
Name: subentry_count, dtype: float64

In [45]:
sorted_entries[sorted_entries.subentry_count == 8].sample(10)

,headword,subentry_count,html
5,अंकुश aṅkuśa,8,"<div> \n\t\t\t\n\t\t\t<hw><d><b>अंकुश</b> <b>aṅkuśa</b></d></hw> <i>m<sup>c</sup></i> (gen. singular) <b>1</b>. (elephant driver's) hook, goad; <d>हाथी को चलाने के लिए</d> ~ <d>मारना</d>/ <d>लगाना</d> thrust a hook in to prod an elephant. <b>2</b>. (<i>fig</i>., <d>नियंत्रण</d>) restraint, control; ~ <d>के नीचे छटपटाना</d> to fret under a restraint; <d>किसी को</d> ~ <d>में रखना</d>, <d>किसी पर</d> ~ <d>रखना</d> to keep smb under control. \n\t\t\t</div>"
9934,से sē,8,"<div> \n\t\t\t\n\t\t\t<hw><d><b>से</b> <b>sē</b></d></hw> <i>post p</i>. <b>1</b>. from <d>मुझे अपने भाई से एक पत्र मिला</d> I received a letter from my brother; <d>सेब कश्मीर से आते हैं</d> apples come from Kashmir; <d>उसने मुझ से चार रुपये उधार में लिये</d> he/she borrowed four rupees from me; <d>हिन्दी से अंग्रेज़ी में अनुवाद करो</d> translate from Hindi into English; <d>वह ज़ुकाम से पीड़ित है</d> he/she is suffering from cold; <d>क्या दफ़्तर यहाँ से दूर है</d> is the office far from here; <d>एक बजे से तीन बजे तक</d> from one to 3'oclock; <d>दिल्ली से कलकत्ता तक</d> from Delhi to Calcutta; <d>आरंभ से अंत तक</d> from beginning to end; <d>एड़ी से चोटी तक</d> from top to bottom. <b>2</b>. with; <d>हम आँखों से देखते हैं</d> we see with our eyes; <d>मैंने कलम से लिखा</d> I wrote with a pen; <d>वह भूख से मरा जा रहा है</d> he is dying with hunger; <d>वह सर्दी से काँप रही थी</d> she was shivering with cold; <d>बच्चा दर्द से कराहने लगा</d> the child started groaning with pain; <d>मैं तुम से सहमत नहीं हूँ</d> I do not agree with you; <d>वह यह काम कठिनाई से कर पाया</d> he could do this work with difficulty; <d>मैं बड़ी ख़ुशी से आऊँगा</d> I shall come with great pleasure; <d>गिलास को सावधानी से रखो</d> place the glass with care or carefully. [Hindi uses <d>से</d> frequently to make adverbs from nouns, as <d>उत्साह से</d> zealously, <d>उत्सुकता से</d> eagerly, <d>तेज़ी से</d> quickly, <d>कड़ाई से</d> sternly, etc.] <b>3</b>. by; <d>हाथ से</d> or <d>डाक से चिट्ठी भेजना</d> to send a letter by hand <d>या</d> by post; <d>वह कार</d> (<d>गाड़ी</d>, <d>बस</d>) <d>से गया</d> he went by car (train, bus); <d>वह जन्म से भारतीय था</d> he was an Indian by birth; <d>वह इस नाम से जाना जाता है</d> he is known by this name; <d>चित्र कील</d> (<d>खूँटी</d>, <d>धागे</d>) <d>से लटक रहा है</d> the picture is hanging by a nail (peg, thread); <d>इंजन बिजली से चलता है</d> the engine is driven by electricity. [most common in passive voice] <d>साँप लड़के से मारा गया</d> the snake was killed by the boy; <d>यह पत्र मुझ से लिखा गया था</d> this letter was written by me; <d>उस से सेब नहीं खाया गया</d> the apple could not be eaten by him. <b>4</b>. [showing degrees of comparison] <d>मैं तुम से बड़ा हूँ</d> I am older than you; <d>वह पहले से अच्छा है</d> he is better than before; <d>यह लड़का सब से अच्छा है</d> this boy is best of all. <b>5</b>. [Misc. usages] <d>मंगलवार से वर्षा हो रही है</d> it has been raining since Tuesday; <d>वह चार महीने से बीमार है</d> he has been ill for four months; <d>लहरें तट से टकरा रही हैं</d> waves are striking against the shore; <d>हम नाक से सूँघते हैं</d> we smell through our nose; <d>हम जीभ से चखते हैं</d> we taste by means of tongue; <d>वह बीमारी या भूख से मरी</d> she died of disease or hunger; <d>किसी चीज़ से डरना</d> to be afraid of smth; <d>किसी से प्यार करना</d> to love smb; <d>मैंने उससे पूछा</d> I asked her/him; <d>लाल स्याही से लिखना</d> to write in red ink; <d>परीक्षा दस बजे से आरम्भ होगी</d> the examination will commence at 10'oclock. <b>6</b>. <d>पर से</d> off; <d>वह घोड़े पर से गिर पड़ा</d> he fell off the horse; <d>उसने मेज़ पर से सब चीज़ें उठा लीं</d> he/she took up all things off the table. \n\t\t\t</div>"
7383,मंच mañca,8,"<div> \n\t\t\t\n\t\t\t<hw><d><b>मंच</b> <b>mañca</b></d></hw> <i>m<sup>c</sup></i> (<i>a</i>) stage, dais [<d>अंतर्राष्ट्रीय</d> international, <d>धार्मिक</d> religious, <d>राजनीतिक</d> political]; ~ <d>पर</d> on the —; ~ <d>पर चढ़ना</d> to ascend the —; ~ <d>पर दिखाना</d> to stage: <d>यही खेल लड़कों व्दारा</d> 

In [44]:
with open('hindi_entries.html', 'w') as outfile:
    for html in sorted_entries[sorted_entries.subentry_count >=8].html:
        outfile.write(f'<div>{html}</div><br/>')
